In [1]:
%%time
import numpy as np
import pandas as pd 

x_train_reduced = pd.read_csv('x_train_reduced.csv')
x_test_reduced = pd.read_csv('x_test_reduced.csv')
y_train2 = pd.read_csv('y_train2.csv')



CPU times: user 2min 9s, sys: 7.06 s, total: 2min 16s
Wall time: 2min 14s


<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


In [2]:
%%time
x_train_reduced = x_train_reduced.loc[:, ~x_train_reduced.columns.str.contains('^Unnamed')]
x_test_reduced = x_test_reduced.loc[:, ~x_test_reduced.columns.str.contains('^Unnamed')]


CPU times: user 690 ms, sys: 624 ms, total: 1.31 s
Wall time: 1.31 s


In [3]:
%%time
y_train3 = y_train2.drop(labels=['fullVisitorId'], axis=1)

CPU times: user 3.9 ms, sys: 0 ns, total: 3.9 ms
Wall time: 3.89 ms


In [12]:
%%time
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional, Dropout
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input
from keras.models import Model

CPU times: user 32 µs, sys: 2 µs, total: 34 µs
Wall time: 37.9 µs


In [13]:
%%time

print(x_train_reduced.shape)
print(y_train3.shape)


(1323730, 290)
(1323730, 1)
CPU times: user 0 ns, sys: 178 µs, total: 178 µs
Wall time: 134 µs


In [14]:
X_train, X_val, y_train, y_val = train_test_split(x_train_reduced, y_train3, test_size=0.15, random_state=666)

In [15]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(1125170, 290)
(198560, 290)
(1125170, 1)
(198560, 1)


In [16]:
# reshape input to be [samples, time steps, features] 
# trainX = np.reshape(X_train, (X_train.shape[0], 1, X_val.shape[1])) 
# testX = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))
X_train = X_train.values
X_val = X_val.values
y_train = y_train.values
y_val = y_val.values
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_val = X_val.reshape(X_val.shape[0],1,X_val.shape[1])

In [17]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(1125170, 1, 290)
(198560, 1, 290)
(1125170, 1)
(198560, 1)


In [18]:
%%time



inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(200,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(120,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model2 = Model(inputs=inputs, outputs=output)
model2.compile(loss='mse', optimizer='adam')
model2.fit(X_train, y_train, epochs=4, batch_size=64, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/4
1125170/1125170 [==============================] - 286s 254us/step - loss: 0.4025 - val_loss: 0.3342
Epoch 2/4
1125170/1125170 [==============================] - 276s 246us/step - loss: 0.2341 - val_loss: 0.2136
Epoch 3/4
1125170/1125170 [==============================] - 276s 245us/step - loss: 0.1867 - val_loss: 0.1834
Epoch 4/4
1125170/1125170 [==============================] - 276s 245us/step - loss: 0.1612 - val_loss: 0.2201
CPU times: user 25min, sys: 3min 50s, total: 28min 50s
Wall time: 18min 39s


In [19]:
y_pred_train = model2.predict(X_train)
y_pred_val = model2.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train))}")

LSTM: RMSE val: 0.46913519472443616  - RMSE train: 0.40283004361167796


In [21]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(200,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(120,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model3 = Model(inputs=inputs, outputs=output)
model3.compile(loss='mse', optimizer='adam')
model3.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/10
1125170/1125170 [==============================] - 286s 254us/step - loss: 0.4038 - val_loss: 0.2538
Epoch 2/10
1125170/1125170 [==============================] - 281s 250us/step - loss: 0.2362 - val_loss: 0.2108
Epoch 3/10
1125170/1125170 [==============================] - 283s 252us/step - loss: 0.1863 - val_loss: 0.1780
Epoch 4/10
1125170/1125170 [==============================] - 283s 252us/step - loss: 0.1617 - val_loss: 0.1542
Epoch 5/10
1125170/1125170 [==============================] - 283s 252us/step - loss: 0.1444 - val_loss: 0.1898
Epoch 6/10
1125170/1125170 [==============================] - 284s 252us/step - loss: 0.1281 - val_loss: 0.1637
Epoch 7/10
1125170/1125170 [==============================] - 283s 251us/step - loss: 0.1221 - val_loss: 0.1718
Epoch 8/10
1125170/1125170 [==============================] - 283s 251us/step - loss: 0.1174 - val_loss: 0.1558
Epoch 9/10
1125170/1125170 [=======================

In [22]:
%%time
y_pred_train = model3.predict(X_train)
y_pred_val = model3.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train))}")

LSTM: RMSE val: 0.40712953991248946  - RMSE train: 0.30902528957293673
CPU times: user 2min 31s, sys: 14.8 s, total: 2min 46s
Wall time: 2min 14s


In [25]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(200,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(120,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model4 = Model(inputs=inputs, outputs=output)
model4.compile(loss='mse', optimizer='adam')
model4.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/20
1125170/1125170 [==============================] - 304s 271us/step - loss: 0.4119 - val_loss: 0.3404
Epoch 2/20
1125170/1125170 [==============================] - 300s 266us/step - loss: 0.2401 - val_loss: 0.2112
Epoch 3/20
1125170/1125170 [==============================] - 300s 267us/step - loss: 0.1891 - val_loss: 0.1837
Epoch 4/20
1125170/1125170 [==============================] - 299s 266us/step - loss: 0.1640 - val_loss: 0.2128
Epoch 5/20
1125170/1125170 [==============================] - 300s 267us/step - loss: 0.1463 - val_loss: 0.1919
Epoch 6/20
1125170/1125170 [==============================] - 299s 265us/step - loss: 0.1346 - val_loss: 0.2132
Epoch 7/20
1125170/1125170 [==============================] - 298s 265us/step - loss: 0.1262 - val_loss: 0.1782
Epoch 8/20
1125170/1125170 [==============================] - 300s 266us/step - loss: 0.1172 - val_loss: 0.1916
Epoch 9/20
1125170/1125170 [=======================

In [27]:
%%time
y_pred_train_4 = model4.predict(X_train)
y_pred_val_4 = model4.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_4))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_4))}")

LSTM: RMSE val: 0.40517249755264917  - RMSE train: 0.25523723436846724
CPU times: user 2min 30s, sys: 14.2 s, total: 2min 44s
Wall time: 2min 12s


In [28]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(200,recurrent_dropout=0.4, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(120,recurrent_dropout=0.4, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model5 = Model(inputs=inputs, outputs=output)
model5.compile(loss='mse', optimizer='adam')
model5.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/10
1125170/1125170 [==============================] - 298s 265us/step - loss: 0.4084 - val_loss: 0.3029
Epoch 2/10
1125170/1125170 [==============================] - 293s 260us/step - loss: 0.2374 - val_loss: 0.1985
Epoch 3/10
1125170/1125170 [==============================] - 293s 261us/step - loss: 0.1905 - val_loss: 0.2086
Epoch 4/10
1125170/1125170 [==============================] - 307s 273us/step - loss: 0.1642 - val_loss: 0.2086
Epoch 5/10
1125170/1125170 [==============================] - 280s 249us/step - loss: 0.1492 - val_loss: 0.1799
Epoch 6/10
1125170/1125170 [==============================] - 293s 260us/step - loss: 0.1400 - val_loss: 0.1644
Epoch 7/10
1125170/1125170 [==============================] - 293s 260us/step - loss: 0.1242 - val_loss: 0.1661
Epoch 8/10
1125170/1125170 [==============================] - 294s 261us/step - loss: 0.1190 - val_loss: 0.1651
Epoch 9/10
1125170/1125170 [=======================

In [29]:
%%time
y_pred_train_5 = model5.predict(X_train)
y_pred_val_5 = model5.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_5))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_5))}")

LSTM: RMSE val: 0.40517249755264917  - RMSE train: 0.25523723436846724
CPU times: user 2min 30s, sys: 14.1 s, total: 2min 45s
Wall time: 2min 13s


In [30]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(200,recurrent_dropout=0.6, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(120,recurrent_dropout=0.6, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model5 = Model(inputs=inputs, outputs=output)
model5.compile(loss='mse', optimizer='adam')
model5.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/10
1125170/1125170 [==============================] - 309s 275us/step - loss: 0.4023 - val_loss: 0.3892
Epoch 2/10
1125170/1125170 [==============================] - 301s 267us/step - loss: 0.2289 - val_loss: 0.2090
Epoch 3/10
1125170/1125170 [==============================] - 302s 268us/step - loss: 0.1848 - val_loss: 0.2322
Epoch 4/10
1125170/1125170 [==============================] - 301s 267us/step - loss: 0.1647 - val_loss: 0.1939
Epoch 5/10
1125170/1125170 [==============================] - 301s 268us/step - loss: 0.1490 - val_loss: 0.1786
Epoch 6/10
1125170/1125170 [==============================] - 301s 267us/step - loss: 0.1352 - val_loss: 0.1642
Epoch 7/10
1125170/1125170 [==============================] - 301s 267us/step - loss: 0.1282 - val_loss: 0.1668
Epoch 8/10
1125170/1125170 [==============================] - 302s 269us/step - loss: 0.1228 - val_loss: 0.1671
Epoch 9/10
1125170/1125170 [=======================

In [32]:
%%time
y_pred_train_6 = model6.predict(X_train)
y_pred_val_6 = model6.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_6))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_6))}")

LSTM: RMSE val: 0.41464140657730414  - RMSE train: 0.3162242457484014
CPU times: user 2min 44s, sys: 19.9 s, total: 3min 4s
Wall time: 2min 22s


In [33]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(180,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(100,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model7 = Model(inputs=inputs, outputs=output)
model7.compile(loss='mse', optimizer='adam')
model7.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/10
1125170/1125170 [==============================] - 299s 265us/step - loss: 0.3969 - val_loss: 0.2675
Epoch 2/10
1125170/1125170 [==============================] - 293s 261us/step - loss: 0.2357 - val_loss: 0.2135
Epoch 3/10
1125170/1125170 [==============================] - 292s 260us/step - loss: 0.1892 - val_loss: 0.2099
Epoch 4/10
1125170/1125170 [==============================] - 292s 260us/step - loss: 0.1637 - val_loss: 0.1994
Epoch 5/10
1125170/1125170 [==============================] - 293s 260us/step - loss: 0.1506 - val_loss: 0.2008
Epoch 6/10
1125170/1125170 [==============================] - 294s 261us/step - loss: 0.1346 - val_loss: 0.1895
Epoch 7/10
1125170/1125170 [==============================] - 292s 260us/step - loss: 0.1230 - val_loss: 0.1646
Epoch 8/10
1125170/1125170 [==============================] - 292s 260us/step - loss: 0.1170 - val_loss: 0.1807
Epoch 9/10
1125170/1125170 [=======================

In [34]:
%%time
y_pred_train_7 = model7.predict(X_train)
y_pred_val_7 = model7.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_7))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_7))}")

LSTM: RMSE val: 0.40063264187181313  - RMSE train: 0.309914904719032
CPU times: user 2min 36s, sys: 14.5 s, total: 2min 51s
Wall time: 2min 20s


In [35]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(200,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(120,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model8 = Model(inputs=inputs, outputs=output)
model8.compile(loss='mse', optimizer='adam')
model8.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/10
1125170/1125170 [==============================] - 610s 542us/step - loss: 0.4149 - val_loss: 0.2779
Epoch 2/10
1125170/1125170 [==============================] - 603s 536us/step - loss: 0.2375 - val_loss: 0.1924
Epoch 3/10
1125170/1125170 [==============================] - 605s 537us/step - loss: 0.1919 - val_loss: 0.2513
Epoch 4/10
1125170/1125170 [==============================] - 603s 536us/step - loss: 0.1695 - val_loss: 0.2000
Epoch 5/10
1125170/1125170 [==============================] - 590s 524us/step - loss: 0.1534 - val_loss: 0.1699
Epoch 6/10
1125170/1125170 [==============================] - 577s 513us/step - loss: 0.1368 - val_loss: 0.1746
Epoch 7/10
1125170/1125170 [==============================] - 578s 513us/step - loss: 0.1326 - val_loss: 0.1754
Epoch 8/10
1125170/1125170 [==============================] - 578s 513us/step - loss: 0.1208 - val_loss: 0.1971
Epoch 9/10
1125170/1125170 [=======================

In [36]:
%%time
y_pred_train_8 = model8.predict(X_train)
y_pred_val_8 = model8.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_8))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_8))}")

LSTM: RMSE val: 0.40779875342955607  - RMSE train: 0.2896994321960738
CPU times: user 2min 31s, sys: 12.8 s, total: 2min 44s
Wall time: 2min 18s


In [37]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(200,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(120,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model9 = Model(inputs=inputs, outputs=output)
model9.compile(loss='mse', optimizer='adam')
model9.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/10
1125170/1125170 [==============================] - 154s 137us/step - loss: 0.4246 - val_loss: 0.2400
Epoch 2/10
1125170/1125170 [==============================] - 147s 131us/step - loss: 0.2322 - val_loss: 0.2186
Epoch 3/10
1125170/1125170 [==============================] - 149s 132us/step - loss: 0.1897 - val_loss: 0.2352
Epoch 4/10
1125170/1125170 [==============================] - 148s 132us/step - loss: 0.1594 - val_loss: 0.1795
Epoch 5/10
1125170/1125170 [==============================] - 147s 131us/step - loss: 0.1432 - val_loss: 0.1942
Epoch 6/10
1125170/1125170 [==============================] - 149s 132us/step - loss: 0.1318 - val_loss: 0.2316
Epoch 7/10
1125170/1125170 [==============================] - 149s 132us/step - loss: 0.1248 - val_loss: 0.1868
Epoch 8/10
1125170/1125170 [==============================] - 147s 131us/step - loss: 0.1125 - val_loss: 0.1650
Epoch 9/10
1125170/1125170 [=======================

In [38]:
%%time
y_pred_train_9 = model9.predict(X_train)
y_pred_val_9 = model9.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_9))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_9))}")

LSTM: RMSE val: 0.39433283022044996  - RMSE train: 0.2933445840252637
CPU times: user 2min 37s, sys: 14.3 s, total: 2min 51s
Wall time: 2min 21s


In [39]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(200,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(120,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model10 = Model(inputs=inputs, outputs=output)
model10.compile(loss='mse', optimizer='adam')
model10.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/10
1125170/1125170 [==============================] - 77s 68us/step - loss: 0.4209 - val_loss: 0.2478
Epoch 2/10
1125170/1125170 [==============================] - 69s 62us/step - loss: 0.2203 - val_loss: 0.2377
Epoch 3/10
1125170/1125170 [==============================] - 69s 62us/step - loss: 0.1790 - val_loss: 0.1804
Epoch 4/10
1125170/1125170 [==============================] - 69s 61us/step - loss: 0.1506 - val_loss: 0.1568
Epoch 5/10
1125170/1125170 [==============================] - 69s 62us/step - loss: 0.1361 - val_loss: 0.1737
Epoch 6/10
1125170/1125170 [==============================] - 69s 62us/step - loss: 0.1288 - val_loss: 0.1714
Epoch 7/10
1125170/1125170 [==============================] - 69s 61us/step - loss: 0.1171 - val_loss: 0.1591
Epoch 8/10
1125170/1125170 [==============================] - 69s 61us/step - loss: 0.1115 - val_loss: 0.1590
Epoch 9/10
1125170/1125170 [==============================] - 69s 6

In [41]:
%%time
y_pred_train_10 = model10.predict(X_train)
y_pred_val_10 = model10.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_10))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_10))}")

LSTM: RMSE val: 0.37683814024328754  - RMSE train: 0.277997960721643
CPU times: user 2min 44s, sys: 25.1 s, total: 3min 9s
Wall time: 2min 15s


In [42]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(200,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(120,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model11 = Model(inputs=inputs, outputs=output)
model11.compile(loss='mse', optimizer='adam')
model11.fit(X_train, y_train, epochs=10, batch_size=300, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/10
1125170/1125170 [==============================] - 68s 60us/step - loss: 0.4375 - val_loss: 0.2405
Epoch 2/10
1125170/1125170 [==============================] - 60s 53us/step - loss: 0.2299 - val_loss: 0.2036
Epoch 3/10
1125170/1125170 [==============================] - 60s 53us/step - loss: 0.1828 - val_loss: 0.1764
Epoch 4/10
1125170/1125170 [==============================] - 60s 53us/step - loss: 0.1598 - val_loss: 0.1956
Epoch 5/10
1125170/1125170 [==============================] - 60s 53us/step - loss: 0.1441 - val_loss: 0.1588
Epoch 6/10
1125170/1125170 [==============================] - 60s 53us/step - loss: 0.1307 - val_loss: 0.1749
Epoch 7/10
1125170/1125170 [==============================] - 60s 53us/step - loss: 0.1229 - val_loss: 0.1509
Epoch 8/10
1125170/1125170 [==============================] - 60s 53us/step - loss: 0.1109 - val_loss: 0.1761
Epoch 9/10
1125170/1125170 [==============================] - 60s 5

In [43]:
%%time
y_pred_train_11 = model11.predict(X_train)
y_pred_val_11 = model11.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_11))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_11))}")

LSTM: RMSE val: 0.424292833978685  - RMSE train: 0.3226801369413016
CPU times: user 2min 47s, sys: 25.1 s, total: 3min 12s
Wall time: 2min 18s


In [44]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(200,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(120,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model12 = Model(inputs=inputs, outputs=output)
model12.compile(loss='mse', optimizer='adam')
model12.fit(X_train, y_train, epochs=20, batch_size=256, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/20
1125170/1125170 [==============================] - 79s 70us/step - loss: 0.4455 - val_loss: 0.3023
Epoch 2/20
1125170/1125170 [==============================] - 71s 63us/step - loss: 0.2290 - val_loss: 0.2220
Epoch 3/20
1125170/1125170 [==============================] - 70s 62us/step - loss: 0.1825 - val_loss: 0.1599
Epoch 4/20
1125170/1125170 [==============================] - 70s 62us/step - loss: 0.1571 - val_loss: 0.1736
Epoch 5/20
1125170/1125170 [==============================] - 70s 62us/step - loss: 0.1370 - val_loss: 0.1622
Epoch 6/20
1125170/1125170 [==============================] - 71s 63us/step - loss: 0.1304 - val_loss: 0.1463
Epoch 7/20
1125170/1125170 [==============================] - 70s 63us/step - loss: 0.1251 - val_loss: 0.1544
Epoch 8/20
1125170/1125170 [==============================] - 70s 63us/step - loss: 0.1116 - val_loss: 0.1829
Epoch 9/20
1125170/1125170 [==============================] - 70s 6

In [45]:
%%time
y_pred_train_12 = model12.predict(X_train)
y_pred_val_12 = model12.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_12))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_12))}")

LSTM: RMSE val: 0.43898293846992925  - RMSE train: 0.3175829332895608
CPU times: user 2min 48s, sys: 24.9 s, total: 3min 12s
Wall time: 2min 19s


In [46]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(200,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(120,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model13 = Model(inputs=inputs, outputs=output)
model13.compile(loss='mse', optimizer='adam')
model13.fit(X_train, y_train, epochs=8, batch_size=256, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/8
1125170/1125170 [==============================] - 80s 71us/step - loss: 0.4297 - val_loss: 0.2559
Epoch 2/8
1125170/1125170 [==============================] - 71s 63us/step - loss: 0.2316 - val_loss: 0.2001
Epoch 3/8
1125170/1125170 [==============================] - 70s 63us/step - loss: 0.1867 - val_loss: 0.1646
Epoch 4/8
1125170/1125170 [==============================] - 70s 63us/step - loss: 0.1579 - val_loss: 0.2106
Epoch 5/8
1125170/1125170 [==============================] - 71s 63us/step - loss: 0.1425 - val_loss: 0.1917
Epoch 6/8
1125170/1125170 [==============================] - 70s 63us/step - loss: 0.1326 - val_loss: 0.1796
Epoch 7/8
1125170/1125170 [==============================] - 71s 63us/step - loss: 0.1198 - val_loss: 0.1802
Epoch 8/8
1125170/1125170 [==============================] - 70s 62us/step - loss: 0.1130 - val_loss: 0.1793
CPU times: user 12min 57s, sys: 2min, total: 14min 57s
Wall time: 9min 38s


In [47]:
%%time
y_pred_train_13 = model13.predict(X_train)
y_pred_val_13 = model13.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_13))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_13))}")

LSTM: RMSE val: 0.42340932501739503  - RMSE train: 0.33514441273062545
CPU times: user 2min 50s, sys: 25.2 s, total: 3min 15s
Wall time: 2min 20s


In [48]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(400,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(240,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model14 = Model(inputs=inputs, outputs=output)
model14.compile(loss='mse', optimizer='adam')
model14.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/10
1125170/1125170 [==============================] - 109s 97us/step - loss: 0.4163 - val_loss: 0.2538
Epoch 2/10
1125170/1125170 [==============================] - 99s 88us/step - loss: 0.2347 - val_loss: 0.2165
Epoch 3/10
1125170/1125170 [==============================] - 98s 87us/step - loss: 0.1904 - val_loss: 0.1811
Epoch 4/10
1125170/1125170 [==============================] - 98s 87us/step - loss: 0.1632 - val_loss: 0.1508
Epoch 5/10
1125170/1125170 [==============================] - 97s 86us/step - loss: 0.1453 - val_loss: 0.1580
Epoch 6/10
1125170/1125170 [==============================] - 97s 86us/step - loss: 0.1321 - val_loss: 0.1712
Epoch 7/10
1125170/1125170 [==============================] - 97s 87us/step - loss: 0.1256 - val_loss: 0.1670
Epoch 8/10
1125170/1125170 [==============================] - 97s 86us/step - loss: 0.1173 - val_loss: 0.1583
Epoch 9/10
1125170/1125170 [==============================] - 97s 

In [49]:
%%time
y_pred_train_14 = model14.predict(X_train)
y_pred_val_14 = model14.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_14))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_14))}")

LSTM: RMSE val: 0.40483751917673755  - RMSE train: 0.28986965367940326
CPU times: user 2min 34s, sys: 16.5 s, total: 2min 50s
Wall time: 2min 8s


In [52]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(180,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(60,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model15 = Model(inputs=inputs, outputs=output)
model15.compile(loss='mse', optimizer='adam')
model15.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/10
1125170/1125170 [==============================] - 80s 71us/step - loss: 0.4284 - val_loss: 0.2415
Epoch 2/10
1125170/1125170 [==============================] - 69s 62us/step - loss: 0.2197 - val_loss: 0.2087
Epoch 3/10
1125170/1125170 [==============================] - 70s 62us/step - loss: 0.1787 - val_loss: 0.1686
Epoch 4/10
1125170/1125170 [==============================] - 69s 62us/step - loss: 0.1564 - val_loss: 0.1724
Epoch 5/10
1125170/1125170 [==============================] - 70s 62us/step - loss: 0.1397 - val_loss: 0.1701
Epoch 6/10
1125170/1125170 [==============================] - 69s 62us/step - loss: 0.1317 - val_loss: 0.1606
Epoch 7/10
1125170/1125170 [==============================] - 70s 62us/step - loss: 0.1185 - val_loss: 0.1581
Epoch 8/10
1125170/1125170 [==============================] - 70s 62us/step - loss: 0.1124 - val_loss: 0.1517
Epoch 9/10
1125170/1125170 [==============================] - 70s 6

In [53]:
%%time
y_pred_train_15 = model15.predict(X_train)
y_pred_val_15 = model15.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_15))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_15))}")

LSTM: RMSE val: 0.40295943441866444  - RMSE train: 0.3073894759202385
CPU times: user 2min 43s, sys: 20 s, total: 3min 3s
Wall time: 2min 18s


In [54]:
%%time

inputs = Input(shape=(1,290))
x = Bidirectional(LSTM(100,recurrent_dropout=0.2, kernel_initializer='lecun_normal', return_sequences=True))(inputs)
x = Bidirectional(LSTM(50,recurrent_dropout=0.2, kernel_initializer='lecun_normal'))(x)
x = Dense(50, activation='sigmoid')(x)
x = Dropout(0.1)(x)
x = Dense(20,activation='elu')(x)
output = Dense(1,activation='linear')(x)

model16 = Model(inputs=inputs, outputs=output)
model16.compile(loss='mse', optimizer='adam')
model16.fit(X_train, y_train, epochs=20, batch_size=256, validation_data=(X_val, y_val), verbose=1, shuffle=False)

Train on 1125170 samples, validate on 198560 samples
Epoch 1/20
1125170/1125170 [==============================] - 79s 70us/step - loss: 0.4274 - val_loss: 0.2491
Epoch 2/20
1125170/1125170 [==============================] - 68s 60us/step - loss: 0.2245 - val_loss: 0.2027
Epoch 3/20
1125170/1125170 [==============================] - 67s 60us/step - loss: 0.1896 - val_loss: 0.1780
Epoch 4/20
1125170/1125170 [==============================] - 68s 60us/step - loss: 0.1669 - val_loss: 0.1771
Epoch 5/20
1125170/1125170 [==============================] - 68s 60us/step - loss: 0.1459 - val_loss: 0.1591
Epoch 6/20
1125170/1125170 [==============================] - 68s 60us/step - loss: 0.1275 - val_loss: 0.1511
Epoch 7/20
1125170/1125170 [==============================] - 68s 60us/step - loss: 0.1221 - val_loss: 0.1719
Epoch 8/20
1125170/1125170 [==============================] - 68s 60us/step - loss: 0.1162 - val_loss: 0.1535
Epoch 9/20
1125170/1125170 [==============================] - 68s 6

In [55]:
%%time
y_pred_train_16 = model16.predict(X_train)
y_pred_val_16 = model16.predict(X_val)

print(f"LSTM: RMSE val: {np.sqrt(mean_squared_error(y_val, y_pred_val_16))}  - RMSE train: {np.sqrt(mean_squared_error(y_train, y_pred_train_16))}")

LSTM: RMSE val: 0.40651840834903064  - RMSE train: 0.279115436234677
CPU times: user 2min 42s, sys: 20.5 s, total: 3min 3s
Wall time: 2min 18s


In [56]:
X_test = x_test_reduced.values
X_test = X_test.reshape((X_test.shape[0],1,X_test.shape[1]))

y_pred_submit = model10.predict(X_test)

In [59]:
test_id = pd.read_csv('test_grouped_id.csv')
test_id = test_id.loc[:, ~test_id.columns.str.contains('^Unnamed')]



/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
y_pred_submit.shape

(296530, 1)

In [64]:
test_id.shape

(296530, 1)

In [66]:
type(test_id)

pandas.core.frame.DataFrame

In [67]:
type(y_pred_submit)

numpy.ndarray

In [81]:
y_pred_submit

array([[0.00305861],
       [0.23765655],
       [0.01008919],
       ...,
       [0.01185551],
       [0.0043706 ],
       [0.00537506]], dtype=float32)

In [82]:
y_pred_submit_reshape = y_pred_submit.reshape(296530)

In [83]:
y_pred_submit_reshape

array([0.00305861, 0.23765655, 0.01008919, ..., 0.01185551, 0.0043706 ,
       0.00537506], dtype=float32)

In [85]:
submission = pd.DataFrame({'fullVisitorId':test_id['fullVisitorId'],'PredictedLogRevenue':y_pred_submit_reshape})
submission['fullVisitorId']= submission['fullVisitorId'].astype(str)
submission['PredictedLogRevenue']=submission['PredictedLogRevenue'].apply(lambda x: 0 if x<0 else x)

submission.to_csv('submission_model10_LSTM.csv',index=False)